In [1]:
from blank_slate_ubi_us.policy import BlankSlatePolicy
from ubicenter import format_fig
from blank_slate_ubi_us.charts.utils import add_numbers
from policyengine.impact.utils import plotly_json_to_fig
from policyengine import PolicyEngineUS

policy = BlankSlatePolicy()

policy.solve()

us = PolicyEngineUS()

baseline, reformed = us.create_microsimulations(policy.reform)

## Table 1

In [2]:
import pandas as pd

change = (
    lambda variable: reformed.calc(variable).sum()
    - baseline.calc(variable).sum()
)

existing_tax_changes = sum(
    [
        baseline.calc(variable).sum()
        for variable in [
            "spm_unit_federal_tax",
            "spm_unit_payroll_tax",
            "spm_unit_self_employment_tax",
        ]
    ]
)

budget = pd.DataFrame(
    {
        "Name": [
            "Abolish federal income tax",
            "Abolish payroll taxes",
            "Flat income tax",
            "Total tax reforms",
            "Repeal SNAP normal allotment",
            "Abolish SSI",
            "Abolish WIC",
            "Abolish TANF",
            "Abolish housing subsidies",
            "Universal basic income",
            "Total benefit reforms",
            "Total tax-benefit reforms",
        ],
        "Revenue effect": [
            -baseline.calc("spm_unit_federal_tax").sum(),
            -baseline.calc("spm_unit_payroll_tax").sum()
            + baseline.calc("spm_unit_self_employment_tax").sum(),
            change("spm_unit_taxes") + existing_tax_changes,
            change("spm_unit_taxes"),
            change("snap"),
            change("ssi"),
            change("wic"),
            change("tanf"),
            change("spm_unit_capped_housing_subsidy"),
            -reformed.calc("basic_income").sum(),
            change("spm_unit_benefits"),
            change("spm_unit_net_income"),
        ],
    }
)
budget["Revenue effect"] = budget["Revenue effect"].apply(
    lambda x: round(x / 1e9)
)
budget

,Name,Revenue effect
0,Abolish federal income tax,-1260
1,Abolish payroll taxes,-582
2,Flat income tax,4377
3,Total tax reforms,2425
4,Repeal SNAP normal allotment,-99
5,Abolish SSI,-62
6,Abolish WIC,-5
7,Abolish TANF,-6
8,Abolish housing subsidies,-24
9,Universal basic income,-2640


## Figure 1

In [3]:
charts = us.population_reform(policy.reform, None)

In [4]:
fig = plotly_json_to_fig(charts["intra_income_decile_chart"]).update_layout(
    title="Distribution of gains and losses by income decile under Blank Slate UBI",
)
for trace in fig.data:
    trace.text = [f"{percent:.0%}" for percent in trace.x]
fig.update_traces(textposition="inside")
fig.update_layout(uniformtext_minsize=15, uniformtext_mode="hide")
format_fig(fig)

## Figure 2

In [5]:
from blank_slate_ubi_us.charts.age_winners import age_winner_chart

fig = plotly_json_to_fig(
    age_winner_chart(baseline, reformed, us.results_config)
).update_layout(
    title="Distribution of gains and losses by age under Blank Slate UBI",
)
for trace in fig.data:
    trace.text = [f"{percent:.0%}" for percent in trace.x]
fig.update_traces(textposition="inside")
fig.update_layout(uniformtext_minsize=15, uniformtext_mode="hide")
format_fig(fig)

## Figure 3

In [6]:
import plotly.express as px
import pandas as pd
from ubicenter import format_fig


def us_state_choropleth(baseline, reformed):
    gain = reformed.calc(
        "spm_unit_net_income", map_to="household"
    ) - baseline.calc("spm_unit_net_income", map_to="household")
    people = pd.Series(
        baseline.calc("person_weight", map_to="household").values
    )
    income = baseline.calc("spm_unit_net_income", map_to="household")
    state = baseline.calc("state_code")
    gain_by_state = gain.groupby(state).sum() / income.groupby(state).sum()
    df = pd.DataFrame(
        {
            "State": gain_by_state.index,
            "Gain": gain_by_state.values,
        }
    )
    df["Label"] = [
        f"On average, people in {state} {'gain' if gain >= 0 else 'lose'} {abs(gain):.1%}"
        for state, gain in zip(df["State"], df["Gain"])
    ]
    fig = px.choropleth(
        locations=gain_by_state.index,
        color=gain_by_state.values,
        locationmode="USA-states",
        scope="usa",
        custom_data=[df.Label],
    )
    fig.update_layout(
        title="Average gain by U.S. State under Blank Slate UBI",
        coloraxis_colorbar_tickformat=".0%",
        coloraxis_colorbar_title="",
        # font_color="white",
    )
    fig.update_traces(
        hovertemplate="%{customdata[0]}",
        marker_line_color="white",  # line markers between states
    )
    return fig


fig = us_state_choropleth(baseline, reformed)

format_fig(fig)

## Figure 4

In [7]:
fig = plotly_json_to_fig(charts["poverty_chart"]).update_layout(
    title="Change to poverty rates by age group under Blank Slate UBI",
)
for trace in fig.data:
    trace.text = [f"{percent:.0%}" for percent in trace.y]
fig.update_traces(textposition="inside")
fig.update_layout(uniformtext_minsize=15, uniformtext_mode="hide")
format_fig(fig)

In [8]:
import numpy as np

poverty_rate = lambda sim: sim.calc(
    "spm_unit_is_in_spm_poverty", map_to="person"
).mean()
deep_poverty_rate = lambda sim: sim.calc(
    "spm_unit_is_in_deep_spm_poverty", map_to="person"
).mean()


def poverty_gap_fn(sim, deep):
    distance = sim.calc("spm_unit_spm_threshold") * (
        1 if not deep else 0.5
    ) - sim.calc("spm_unit_net_income")
    positive_distance = np.maximum(0, distance)
    weight = sim.calc("spm_unit_weight").values
    return (weight * positive_distance).sum()


poverty_gap = lambda sim: poverty_gap_fn(sim, False)
deep_poverty_gap = lambda sim: poverty_gap_fn(sim, True)
metric_names = [
    "Poverty rate",
    "Deep poverty rate",
    "Poverty gap",
    "Deep poverty gap",
]

import pandas as pd

# For each metric, add columns "Name", "Baseline", "Reformed", "Absolute change", "Relative change".

df = pd.DataFrame(
    {
        "Name": metric_names,
        "Baseline": [
            poverty_rate(baseline) / 1e-2,
            deep_poverty_rate(baseline) / 1e-2,
            poverty_gap(baseline) / 1e9,
            deep_poverty_gap(baseline) / 1e9,
        ],
        "Reformed": [
            poverty_rate(reformed) / 1e-2,
            deep_poverty_rate(reformed) / 1e-2,
            poverty_gap(reformed) / 1e9,
            deep_poverty_gap(reformed) / 1e9,
        ],
    }
)

df["Change"] = df.Reformed - df.Baseline
df["Relative change"] = (df.Reformed / df.Baseline - 1) * 100

df.Baseline = df.Baseline.round(1)
df.Reformed = df.Reformed.round(1)
df.Change = df.Change.round(1)
df["Relative change"] = df["Relative change"].round(1)

df

,Name,Baseline,Reformed,Change,Relative change
0,Poverty rate,11.2,5.5,-5.8,-51.4
1,Deep poverty rate,3.2,0.5,-2.6,-82.9
2,Poverty gap,130.1,55.9,-74.2,-57.0
3,Deep poverty gap,23.2,4.1,-19.2,-82.5


In [9]:
fig = plotly_json_to_fig(charts["rel_income_decile_chart"]).update_layout(
    title="Change to net income by income decile under Blank Slate UBI",
)
for trace in fig.data:
    trace.text = [f"{percent:.0%}" for percent in trace.y]
fig.update_traces(textposition="inside")
fig.update_layout(uniformtext_minsize=15, uniformtext_mode="hide")
format_fig(fig)

In [10]:
fig = plotly_json_to_fig(charts["inequality_chart"]).update_layout(
    title="Changes to inequality metrics under Blank Slate UBI",
)
for trace in fig.data:
    trace.text = [f"{percent:.0%}" for percent in trace.y]
fig.update_traces(textposition="inside")
fig.update_layout(uniformtext_minsize=15, uniformtext_mode="hide")
format_fig(fig)

In [11]:
import plotly.express as px
import pandas as pd
from ubicenter import format_fig


def us_state_poverty_choropleth(baseline, reformed):
    state = baseline.calc("state_code", map_to="person")
    baseline_poverty = (
        baseline.calc("spm_unit_is_in_spm_poverty", map_to="person")
        .groupby(state)
        .mean()
    )
    reform_poverty = (
        reformed.calc("spm_unit_is_in_spm_poverty", map_to="person")
        .groupby(state)
        .mean()
    )
    rel_change = reform_poverty / baseline_poverty - 1
    df = pd.DataFrame(
        {
            "State": rel_change.index,
            "Gain": rel_change.values,
        }
    )
    df["Label"] = [
        f"The poverty rate in {state} {'increases' if gain >= 0 else 'falls'} by {abs(gain):.1%}"
        for state, gain in zip(df["State"], df["Gain"])
    ]
    fig = px.choropleth(
        locations=rel_change.index,
        color=-rel_change.values,
        locationmode="USA-states",
        scope="usa",
        custom_data=[df.Label],
    )
    fig.update_layout(
        title="Poverty rate reduction by U.S. State under Blank Slate UBI",
        coloraxis_colorbar_tickformat=".0%",
        coloraxis_colorbar_title="",
    )
    fig.update_traces(
        hovertemplate="%{customdata[0]}",
        marker_line_color="white",  # line markers between states
    )
    return fig


fig = us_state_poverty_choropleth(baseline, reformed)
format_fig(fig)

In [12]:
from blank_slate_ubi_us.charts.program_winners import program_winner_chart

fig = program_winner_chart(baseline, reformed, us.results_config)
for trace in fig.data:
    trace.text = [f"{percent:.0%}" for percent in trace.x]
fig.update_traces(textposition="inside")
fig.update_layout(uniformtext_minsize=15, uniformtext_mode="hide")
format_fig(fig)

In [13]:
household = {
    "people": {
        "adult": {
            "age": {2022: 25},
        },
        "adult_2": {
            "age": {2022: 25},
        },
        "child": {
            "age": {2022: 5},
        },
    },
    "tax_units": {
        "tax_unit": {
            "members": ["adult", "adult_2", "child"],
        }
    },
    "marital_units": {
        "marital_unit": {
            "members": ["adult", "adult_2"],
        }
    },
    "spm_units": {
        "spm_unit": {
            "members": ["adult", "adult_2", "child"],
        }
    },
    "households": {
        "household": {
            "members": ["adult", "adult_2", "child"],
            "state_code": {2022: "MA"},
            "zip_code": {2022: "01001"},
        }
    },
}

fig = us.household_variation(dict(**policy.reform, household=household))

In [14]:
import plotly.io as pio

pio.renderers.default = "browser"
f = plotly_json_to_fig(fig["budget_chart"]).update_layout(
    title="Net income by employment income under Blank Slate UBI",
)
format_fig(f)

In [15]:
format_fig(
    plotly_json_to_fig(fig["mtr_chart"]).update_layout(
        title="Marginal tax rates under Blank Slate UBI",
        yaxis_range=(-0.5, 1),
    )
)

Opening in existing browser session.


Opening in existing browser session.
